In [65]:
import requests
import json
import string
import pickle
base_url = "https://api.github.com"
# https://github.com/settings/tokens
api_key = ""
save_folder = f"./local_save/"
with open('api.json', 'r') as file:
    api_key = json.load(file)["GH_API_KEY"]
    
def pickle_this(target, name, file_path=save_folder):
    path = file_path+name
    file = open(path, "wb")
    if file:
        pickle.dump(target, file)
        file.close()
        return True
    file.close()
    return False
    

def unpickle_this(name, file_path=save_folder):
    path = file_path+name
    file = open(path, "rb")
    res = []
    if file:
        res = pickle.load(file)
        file.close()
        return res
    file.close()
    return res


def get_call(paths, params=None, header=None):
    url = f"{base_url}/{paths}"
    response = requests.get(url, params=params, headers=header)
    if response.status_code == 200:
        return response.json()
    elif response.status_code == 403:
        print("La patience est d'or. Enculé")
        sleep(3600)
        response = requests.get(url, params=params, headers=header)
        return response.json()
    else:
        print(f"Error: {response.status_code} - {response.json().get('message', 'Unknown error')}")
        return {}


def get_repos_created_in_year(year, month_start="01", month_end="12", day_start="01", day_end="31"):
    # Recherche de repositories créés en 2024
    query = "created:>="+str(year)+"-"+str(month_start)+"-"+str(day_start)+" created:<="+str(year)+"-"+str(month_end)+"-"+str(day_end)
    params = {"q": query, "per-page": 1}
    header = {"Authorization": f"token {api_key}"}
    # Appeler l'API de recherche de repositories
    repos_data = get_call("search/repositories", params=params, header=header)
    if repos_data: 
        return repos_data.get("items", [])
    else:
        return None
    

def get_repos_created_in_day_multipages(year, month="01", day="01", max_pages=0):
    # Recherche de repositories créés en 2024
    query = "created:"+str(year)+"-"+str(month)+"-"+str(day)
    params = {"q": query, "per-page": 30}
    header = {"Authorization": f"token {api_key}"}
    # Appeler l'API de recherche de repositories
    repos_data = get_call("search/repositories", params=params, header=header)
    num_pages = 0
    res = []
    if repos_data:
        res.extend(repos_data.get("items", []))
        repo_pages = (repos_data.get("total_count", 0) // 30)
        print(repo_pages)
        if max_pages > 0:
            num_pages = min(max_pages, repo_pages)
        
        for ii in range(2, num_pages+1):
            repos_data={}
            params = {"q": query, "per-page": 30, "page": ii}
            repos_data = get_call("search/repositories", params=params, header=header)
            res.extend(repos_data.get("items", []))
            pickle_this(res, "repos_"+str(year)+"_"+str(month))
        
        return res
    else:
        return []
    
    
def get_languages_from_repositories(repositories):
    # Extraire la liste des langages
    return {repo["language"] for repo in repositories if repo["language"]}

    
def print_languages_from_repositories(repositories):
    # Extraire la liste des langages
    languages = {repo["language"] for repo in repositories if repo["language"]}

    # Afficher les langages trouvés
    if languages:
        print("Langages utilisés dans les repositories créés en 2024 :")
        for language in sorted(languages):
            print(f"- {language}")
    else:
        print("Aucun langage trouvé dans les repositories.")

In [67]:
res = get_repos_created_in_year_multipages(year=2024, max_pages=50)
print(len(res))

117806
Error: 422 - Only the first 1000 search results are available


AttributeError: 'NoneType' object has no attribute 'get'

In [ ]:
for ii in range(1, 10):
    print(ii)

2
3
4
5
6
7
8
9


In [73]:
# Récupérer les repositories créés en 2024
repositories = get_test(year=2024)
languages = []
# Analyser les langages utilisés
if repositories:
    #print(repositories[0])
    languages = get_languages_from_repositories(repositories)
else:
    print("Aucun repository trouvé pour 2024.")

print(languages)

TypeError: 'Response' object is not subscriptable

In [67]:
print(repositories[0].keys())

IndexError: list index out of range

In [9]:
pickle_this(repositories, "repos")

True

In [7]:
results = unpickle_this("repos")

In [13]:
for ii in range(2010, 2026):
    if ii in dict.keys():
        repositories = get_repos_created_in_year(year=ii)
        # Analyser les langages utilisés
        if repositories:
            pickle_this(repositories, "repos"+ii)
            results[str(ii)] = get_languages_from_repositories(repositories)
        else:
            print(f"Aucun repository trouvé pour %n.", ii)
print(results)

TypeError: unbound method dict.keys() needs an argument

In [14]:
len(results)

30

In [45]:
def get_last_page_link(link_str):
        link_last = ""
        num_last = 1
    for page in response["link"].split(", "):
        page_split = page.split("; ")
        link_last = page_split[0][1:-1]
        if "last" in page_split[1]:
            num_last = link_last.split("&page=")
    link_last = link_last+"&page="
    return [link_last, num_last]

['https://api.github.com/search/repositories?q=created%3A%3E%3D2024-01-01+created%3A%3C%3D2024-12-31&per_page=15000000', '2']
